In [1]:
# This notebook is to try and build a tool for the agents to use

In [2]:
from dataclasses import dataclass
from decimal import Decimal
from typing import List, Literal, Optional, Dict, Union

In [7]:
Country = Literal["UK", "US", "CA"]

@dataclass
class Location:
    country: Country                      # "US", "UK", "CA"
    region: Optional[str] = None          # e.g. "NY", "BC"; None for UK

@dataclass
class Heir:
    relationship: Literal["child", "spouse", "other"]
    share_of_estate: Decimal              # % or £/$ value – decide which you prefer
    location: Location

@dataclass
class LifetimeGift:
    date: str
    amount: Decimal
    gift_tax_paid: Decimal = Decimal("0")
    gift_type: Literal["PET", "CLT"] = "PET"

@dataclass
class EstateInput:
    estate_location: Location             # <- NEW (replaces earlier jurisdiction/state)
    heirs: List[Heir]                     # <- NEW
    date_of_death: str
    gross_estate: Dict[str, Decimal]
    debts_and_costs: Decimal = Decimal("0")
    marital_status: Literal["single", "married"] = "single"
    spouse_citizenship_us: Optional[bool] = None
    was_predeceased_spouse: bool = False
    transferred_allowances: Dict[str, Decimal] = None
    charity_bequest_percent: Decimal = Decimal("0")
    qualifying_home_value: Decimal = Decimal("0")
    passed_to_direct_descendants: bool = False
    bpr_value: Decimal = Decimal("0")
    apr_value: Decimal = Decimal("0")
    special_use_farm_value: Decimal = Decimal("0")
    lifetime_gifts: List[LifetimeGift] = None
    capital_gains_at_death: Optional[Dict[str, Decimal]] = None
    charitable_donation_current_year: Decimal = Decimal("0")
    currency_fx_rates: Dict[str, Decimal] = None

In [8]:
# Create a 'top-level dispatcher' function to simply look at the country on our EstateInput
# and then send the estate to the right 'sub-calculator'

def calculate_inheritance_tax(data: EstateInput):
    country = data.estate_location.country
    if country == "UK":
        return _uk_iht(data)
    elif country == "US":
        return _us_estate_tax(data)          # <- will read data.estate_location.region
    elif country == "CA":
        return _ca_death_tax(data)
    else:
        raise ValueError("Unsupported country")

In [13]:
# --- US Estate Tax Constants & Functions ---

# 2025 Basic Exclusion Amount (federal)
BEA = Decimal("13990000")

# 2025 Federal estate tax brackets (threshold, rate)
US_ESTATE_BRACKETS = [
    (Decimal("0"),       Decimal("0.18")),
    (Decimal("10000"),   Decimal("0.20")),
    (Decimal("20000"),   Decimal("0.22")),
    (Decimal("40000"),   Decimal("0.24")),
    (Decimal("60000"),   Decimal("0.26")),
    (Decimal("80000"),   Decimal("0.28")),
    (Decimal("100000"),  Decimal("0.30")),
    (Decimal("150000"),  Decimal("0.32")),
    (Decimal("250000"),  Decimal("0.34")),
    (Decimal("500000"),  Decimal("0.37")),
    (Decimal("1000000"), Decimal("0.40")),
]

# State‐level estate tax (flat rates example)
STATE_ESTATE_TAX_RATES: Dict[str, Decimal] = {
    "MA": Decimal("0.16"),
    "NY": Decimal("0.16"),
}

# State inheritance‐tax rates by heir relationship (example)
STATE_INHERITANCE_TAX_RATES: Dict[str, Dict[str, Decimal]] = {
    "PA": {"spouse": Decimal("0.04"), "child": Decimal("0.04"), "other": Decimal("0.15")}
}

def _federal_estate_layer(data) -> Dict[str, Decimal]:
    net = data.gross_estate.get("USD", Decimal("0")) - data.debts_and_costs
    taxable = max(Decimal("0"), net - BEA)
    tax_due = Decimal("0")
    for i, (threshold, rate) in enumerate(US_ESTATE_BRACKETS):
        if taxable <= threshold:
            break
        next_threshold = (
            US_ESTATE_BRACKETS[i+1][0]
            if i+1 < len(US_ESTATE_BRACKETS)
            else taxable
        )
        portion = min(taxable, next_threshold) - threshold
        tax_due += portion * rate
    return {
        "tax_due": tax_due.quantize(Decimal("0.01")),
        "taxable_base": net
    }

def _state_estate_layer(taxable_estate: Decimal, state: str) -> Decimal:
    rate = STATE_ESTATE_TAX_RATES.get(state, Decimal("0"))
    return (taxable_estate * rate).quantize(Decimal("0.01"))

def _us_estate_tax(data) -> Dict[str, Any]:
    fed = _federal_estate_layer(data)
    state = data.estate_location.region or ""
    state_tax = _state_estate_layer(fed["taxable_base"], state)
    inh_tax = Decimal("0")
    if state in STATE_INHERITANCE_TAX_RATES:
        for heir in data.heirs:
            r = STATE_INHERITANCE_TAX_RATES[state].get(heir.relationship, Decimal("0"))
            inh_tax += (heir.share_of_estate * r).quantize(Decimal("0.01"))
    total = fed["tax_due"] + state_tax + inh_tax
    return {
        "tax_due": total.quantize(Decimal("0.01")),
        "breakdown": {"federal": fed, "state_estate": state_tax, "state_inheritance": inh_tax},
        "currency": "USD"
    }

In [10]:
# --- UK IHT with PET/CLT taper ---

NRB = Decimal("325000")
RNRB = Decimal("175000")
RNRB_TAPER_THRESHOLD = Decimal("2000000")

def _calculate_pet_tax(data) -> Decimal:
    death = datetime.fromisoformat(data.date_of_death)
    total = Decimal("0")
    for gift in data.lifetime_gifts or []:
        if gift.gift_type != "PET":
            continue
        gdt = datetime.fromisoformat(gift.date)
        years = (death - gdt).days / 365.0
        if years >= 7:
            continue
        if years < 3:
            factor = Decimal("1")
        elif years < 4:
            factor = Decimal("0.80")
        elif years < 5:
            factor = Decimal("0.60")
        elif years < 6:
            factor = Decimal("0.40")
        else:
            factor = Decimal("0.20")
        total += gift.amount * Decimal("0.40") * factor
    return total.quantize(Decimal("0.01"))

def _uk_iht(data) -> Dict[str, Any]:
    net = data.gross_estate.get("GBP", Decimal("0")) - data.debts_and_costs
    net_reliefs = net - data.bpr_value - data.apr_value

    pct = (data.transferred_allowances or {}).get("nrb_pct", Decimal("0"))
    nrb_avail = NRB + (pct * NRB)

    rnrb_avail = Decimal("0")
    if data.passed_to_direct_descendants:
        rnrb_avail = RNRB
        if net > RNRB_TAPER_THRESHOLD:
            taper = (net - RNRB_TAPER_THRESHOLD) / 2
            rnrb_avail = max(Decimal("0"), RNRB - taper)

    chargeable = max(Decimal("0"), net_reliefs - (nrb_avail + rnrb_avail))
    rate = Decimal("0.36") if data.charity_bequest_percent >= Decimal("10") else Decimal("0.40")
    base_tax = (chargeable * rate).quantize(Decimal("0.01"))

    pet_tax = _calculate_pet_tax(data)
    clt_credit = sum(g.gift_tax_paid for g in data.lifetime_gifts or [])

    total = base_tax + pet_tax - Decimal(clt_credit)
    eff = (
        (total / net_reliefs * Decimal("100")).quantize(Decimal("0.01"))
        if net_reliefs > 0 else Decimal("0.00")
    )

    return {
        "tax_due": total,
        "effective_rate": f"{eff}%",
        "breakdown": {
            "net_after_reliefs": net_reliefs,
            "nrb_available": nrb_avail,
            "rnrb_available": rnrb_avail,
            "chargeable_estate": chargeable,
            "base_iht": base_tax,
            "pet_iht": pet_tax,
            "clt_credit": Decimal(clt_credit)
        },
        "currency": "GBP"
    }

In [14]:
# --- Canada probate fee & CGT at death ---

PROBATE_FEE_RATES: Dict[str, Decimal] = {
    "ON": Decimal("0.015"),
    "BC": Decimal("0.014"),
}

def _ca_death_tax(data) -> Dict[str, Any]:
    net = data.gross_estate.get("CAD", Decimal("0")) - data.debts_and_costs
    prov = data.estate_location.region or ""
    fee = (net * PROBATE_FEE_RATES.get(prov, Decimal("0.015"))).quantize(Decimal("0.01"))

    cg = data.capital_gains_at_death or {}
    gains = cg.get("taxable_gains", Decimal("0"))
    losses = cg.get("unused_net_capital_losses", Decimal("0"))
    death = datetime.fromisoformat(data.date_of_death)
    incl = Decimal("0.667") if death >= datetime(2024,6,25) else Decimal("0.50")
    taxable_cg = max(Decimal("0"), gains * incl - losses).quantize(Decimal("0.01"))

    return {
        "tax_due": fee,
        "breakdown": {"probate_fee": fee, "taxable_capital_gains": taxable_cg},
        "currency": "CAD"
    }

In [15]:
# This cell is to test the calculators above.

from decimal import Decimal

# --- UK test ---
uk_input = EstateInput(
    estate_location=Location(country="UK"),
    heirs=[],  # empty for now
    date_of_death="2025-05-04",
    gross_estate={"GBP": Decimal("750000")},
    debts_and_costs=Decimal("50000"),
    bpr_value=Decimal("50000"),      # business relief
    apr_value=Decimal("0"),          # no agricultural relief
    passed_to_direct_descendants=True,
    transferred_allowances={"nrb_pct": Decimal("0"), "rnrb_pct": Decimal("0")},
    charity_bequest_percent=Decimal("12")  # triggers 36% rate
)

# --- Canada test ---
ca_input = EstateInput(
    estate_location=Location(country="CA", region="BC"),
    heirs=[],
    date_of_death="2025-05-04",
    gross_estate={"CAD": Decimal("1200000")},
    debts_and_costs=Decimal("200000")
)

# Run and print
print("UK IHT Result:")
print(calculate_inheritance_tax(uk_input))
print("\nCanada Probate Fee Result:")
print(calculate_inheritance_tax(ca_input))

UK IHT Result:
{'tax_due': Decimal('54000.00'), 'effective_rate': '8.31%', 'breakdown': {'net_after_reliefs': Decimal('650000'), 'nrb_available': Decimal('325000'), 'rnrb_available': Decimal('175000'), 'chargeable_estate': Decimal('150000')}, 'currency': 'GBP'}

Canada Probate Fee Result:


NameError: name 'datetime' is not defined